# Watershed segmentation for cell identification

In [1]:
from math import sqrt

import numpy as np

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import skimage
from skimage import io, exposure, img_as_float, img_as_ubyte, morphology, filters, util
from skimage.color import rgb2gray, label2rgb
from skimage.feature import canny, blob_dog, blob_log, blob_doh,  peak_local_max
from skimage.filters import sobel, threshold_otsu, try_all_threshold, threshold_local, threshold_minimum
from skimage.segmentation import clear_border, watershed, random_walker
from skimage.measure import label, regionprops
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.morphology import closing

import cv2

from scipy import ndimage as ndi

import os

import plotly.graph_objs as go

### Read image

Read image as greyscale and color in parallel

In [ ]:
image_file = '/Users/carolinemccue/Documents/GitHub/CrystalML/notebooks/MG63/r01c01f01p01-ch2sk1fk1fl1.tiff'
f_grey = cv2.imread(image_file, 0) # 0 reads as greyscale
f_color = cv2.imread(image_file, 1) # reads as color
plt.figure(0)
plt.imshow(f_grey, cmap=plt.cm.gray)

In [ ]:
f_grey

### Crop area of interest

and convert to uint (for some skimage methods)

In [ ]:
f_cropped = f_grey[0:1400,0:2000] #[lines, columns]
f_color_cropped = f_color[0:1400,0:2000] #[lines, columns]
plt.figure(2)
plt.imshow(f_cropped, cmap=plt.cm.gray, vmin=0, vmax=255)

### Increase contrast

Adaptive equalization can deal with some non-uniformity in the original image

In [ ]:
# Adaptive Equalization
clahe = cv2.createCLAHE(clipLimit=15, tileGridSize=(8,8))
img_adapteq = clahe.apply(f_cropped)
plt.imshow(img_adapteq, cmap=plt.cm.gray)

### Threshold

Otsu threshold was found to work best in this type of image (see SegmentationDropletsInCapillary).

In [ ]:
image = img_adapteq

blur = cv2.GaussianBlur(image,(5,5),0)
retVal, binary = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

fig, axes = plt.subplots(ncols=3, figsize=(22, 6))
ax = axes.ravel()
ax[0] = plt.subplot(1, 3, 1)
ax[1] = plt.subplot(1, 3, 2)
ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original')
ax[0].axis('off')

ax[1].hist(image.ravel(), bins=256)
ax[1].set_title('Histogram')
ax[1].axvline(retVal, color='r')

ax[2].imshow(binary, cmap=plt.cm.gray)
ax[2].set_title('Thresholded')
ax[2].axis('off')

print('Minimum threshold: ' + str(retVal))
print('Min intensity: ' + str(np.min(image)))
print('Max intensity: ' + str(np.max(image)))
plt.show()

### Perform Watershed segmentation

- Close to remove artifacts from contrast increase and thresholding
- Detect sure foreground and background from thresholding and distance transform
- Run the watershed algorithm to segment
- Label segmented regions

#### Closing and removing small dark spots from crystal borders

In [ ]:
from skimage.morphology  import binary_closing, disk, binary_dilation, remove_small_holes

# Remove dark spots and connect bright spots
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations = 1)

# Fill holes 
#fill_holes = ndi.morphology.binary_fill_holes(closed, structure=np.ones((3, 3))).astype('uint8')

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 15))
ax = axes.ravel()

ax[0].imshow(binary, cmap=plt.cm.gray, interpolation='nearest')
ax[0].set_title('Binary')
ax[1].imshow(closed, cmap=plt.cm.gray, interpolation='nearest')
ax[1].set_title('Closed')
#ax[2].imshow(fill_holes, cmap=plt.cm.gray, interpolation='nearest')
#ax[2].set_title('Closed + Fill holes')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()
plt.savefig("2_thresh_close.jpg")

#### Sure foreground / background determination

In [ ]:
def clear_border(image):
    h, w = image.shape
    mask = np.zeros((h + 2, w + 2), np.uint8)
    for i in range(h-1): # Iterate on the lines
        if image[i, 0] == 255:
            cv2.floodFill(image, mask, (0, i), 0)
        if image[i, w-1] == 255:
            cv2.floodFill(image, mask, (w-1, i), 0)
    for i in range(w-1): # Iterate on the columns
        if image[0, i] == 255:
            cv2.floodFill(image, mask, (i, 0), 0)
        if image[h-1, i] == 255:
            cv2.floodFill(image, mask, (i, h-1), 0)
    return image

In [ ]:
# Alternative thresholding

In [ ]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(closed,cv2.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
cell_outline = cv2.subtract(sure_bg,sure_fg)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))
ax = axes.ravel()

ax[0].imshow(sure_bg, cmap=plt.cm.gray, interpolation='nearest')
ax[0].set_title('Sure Background')
ax[1].imshow(sure_fg, cmap=plt.cm.gray, interpolation='nearest')
ax[1].set_title('Sure foreground')
#ax[2].imshow(-mask, cmap=plt.cm.gray, interpolation='nearest')
#ax[2].set_title('Mask')
ax[2].imshow(cell_outline, cmap=plt.cm.gray, interpolation='nearest')
ax[2].set_title('Cell Outline')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
# sure background area
#kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(4,4))
#dilated = cv2.dilate(closed,kernel,iterations=1)

##dist_transform = cv2.distanceTransform(fill_holes,cv2.DIST_L2,5)

##sure_bg = fill_holes.copy()
##sure_bg[(dist_transform != 0) & (dist_transform < 0.1*dist_transform.max())] = 0

#sure_bg = np.uint8(dilated)

## Finding sure foreground area
#dist_transform_fg = cv2.distanceTransform(closed,cv2.DIST_L2,5)
#ret, sure_fg = cv2.threshold(dist_transform_fg,0.2*dist_transform_fg.max(),255,0)
#clear_border(sure_fg)

## Masking
#mask = dist_transform_fg.copy()
#mask[mask < 10] = 0

## Finding unknown region
#sure_fg = np.uint8(sure_fg)
#cell_outline = cv2.subtract(sure_bg,sure_fg)

#fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))
#ax = axes.ravel()

#ax[0].imshow(sure_bg, cmap=plt.cm.gray, interpolation='nearest')
#ax[0].set_title('Sure Background')
#ax[1].imshow(sure_fg, cmap=plt.cm.gray, interpolation='nearest')
#ax[1].set_title('Sure foreground')
#ax[2].imshow(-mask, cmap=plt.cm.gray, interpolation='nearest')
#ax[2].set_title('Mask')
#ax[3].imshow(cell_outline, cmap=plt.cm.gray, interpolation='nearest')
#ax[3].set_title('Cell Outline')

#for a in ax:
    #a.set_axis_off()

#plt.tight_layout()
#plt.show()

#### Label markers

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
#markers = markers+1

# Now, mark the region of unknown with zero
#markers[sure_bg] = 1

fig = plt.figure(figsize=(15,10))
plt.imshow(markers, cmap=plt.cm.tab20)

#### Run the watershed algorithm

In [ ]:
threeChannels = cv2.cvtColor(closed, cv2.COLOR_GRAY2BGR)
segmented = cv2.watershed(threeChannels.astype('uint8'), markers)

# Color the labels sections for display
image_label_overlay = label2rgb(segmented, image=img_adapteq, bg_label=1)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 15))
ax = axes.ravel()

ax[0].imshow(closed, cmap=plt.cm.gray, interpolation='nearest')
ax[0].contour(segmented, [0.5], linewidths=1, colors='r')
ax[0].set_title('Contour')
ax[1].imshow(image_label_overlay)
ax[1].set_title('Segmented Image')

for a in ax:
    a.set_axis_off()

    
plt.tight_layout()
plt.show()

#### Extract the region properties

Note that label 1 is attributed to the background in the code above

In [ ]:
reg = regionprops(segmented, intensity_image=f_cropped)[1:]
columns= ['Label', 'Area', 'Eccentricity', 'Circularity', 'Bbox Area', 'Bbox', 'Centroid', 'Image']
df = pd.DataFrame(columns=columns, dtype=np.float64)
df = df.append([{'Label':i.label, 'Area':i.area, 'Eccentricity':i.eccentricity, 'Circularity':(4 * np.pi * i.area / (i.perimeter**2)), 'Bbox Area':i.bbox_area, 'Bbox':i.bbox, 'Centroid':i.centroid, 'Image':i.intensity_image} for i in reg])
df.head(5)

#### Display labeled region bounding boxes and labels

In [ ]:
boxed_regions = f_cropped.copy()

# Iterate over all non-background labels
for region in reg:
    (min_row, min_col, max_row, max_col) = region.bbox
    color = np.uint8(np.random.randint(0, 255, 3)).tolist()
    cv2.rectangle(boxed_regions, (min_col,min_row), (max_col,max_row), color, 3)
    cv2.putText(boxed_regions,'%d'%region.label,(min_col+50, min_row+100), cv2.FONT_HERSHEY_SIMPLEX, 3, color, 3, cv2.LINE_AA)
    
fig = plt.figure(figsize=(15,10))
plt.imshow(boxed_regions, cmap=plt.cm.gray)

#### Evaluate peo

In [ ]:
df[df['Label'] == 88]

In [ ]:
df.Circularity.plot()

In [ ]:
df.Eccentricity.plot()

In [ ]:
df['Area'].plot()

In [ ]:
plt.scatter(df.Eccentricity,df["Area"])

In [ ]:
plt.scatter(df.Eccentricity,df["Bbox Area"])

In [ ]:
# Display parameters
org_img = f_cropped # Intensity image
border = 15 # Additional pixels to display on each side of the bounding box
#ecc_cutoff = 0.8 # Eccentricity cuttoff, removes misidentified shapes (for instance half-droplets)
area_cutoff = 0.75*df.Area.mean() # Area cuttoff, removes misidentified shapes (for instance half-droplets)
max_col = org_img.shape[1]
max_row = org_img.shape[0]

n_img = len(reg)
print("{} cells were detected and segmented".format(n_img))
n_col = 15
n_rows = n_img // n_col + 1

# Sort as a function of eccentricity
reg.sort(key=lambda x: x.area)
reg_clean = [region for region in reg if (region.area) > area_cutoff]

print("{} cells with selected criteria".format(len(reg_clean), area_cutoff))

fig, ax = plt.subplots(n_col, n_rows)
axes = ax.ravel()

for i, region in enumerate(reg_clean):
    (min_row, min_col, max_row, max_col) = region.bbox
    drop_image = org_img[np.max([min_row-border,0]):np.min([max_row+border,max_row]),np.max([min_col-border,0]):np.min([max_col+border,max_col])]
    contrast_stretch = exposure.rescale_intensity(drop_image, in_range=(0,255))
    axes[i].imshow(contrast_stretch, cmap=plt.cm.gray, interpolation='nearest', vmin=0, vmax=255)
    axes[i].axis('off')
    
for i in range(n_col*n_rows - len(reg_clean)):
    axes[-(i+1)].axis('off')
    
fig.set_size_inches(15, 25)